In [1]:
from dotenv import find_dotenv, load_dotenv
from transformers import pipeline
import requests
import os
import streamlit as st

load_dotenv(find_dotenv())

True

In [2]:
HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [3]:
# img2txt
def img2txt(url):
    image_to_text = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", max_new_tokens=50)
    text = image_to_text(url)[0]["generated_text"]
    print(text)
    return text

In [4]:
senario = img2txt("maryna-vasylieva-vasilievame-2xDIrOkhEb0-unsplash.jpg")

there are coffee bags and a cup of coffee on a table


In [5]:
# llm
def generate_story(senario):
    API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_TOKEN}"}
    def query(payload):
    	response = requests.post(API_URL, headers=headers, json=payload)
    	return response.json()
	
    output = query({
	"inputs": "<s>[INST] write short story about " + senario + "[/INST]",
    })
    print(output[0]["generated_text"])
    return output



In [11]:
generate_story(senario)

<s>[INST] write short story about there are coffee bags and a cup of coffee on a table[/INST] Once upon a time, there was a small coffee shop on the corner of the street. The shop


[{'generated_text': '<s>[INST] write short story about there are coffee bags and a cup of coffee on a table[/INST] Once upon a time, there was a small coffee shop on the corner of the street. The shop'}]

In [34]:
def new_story(senario):
    API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_TOKEN}", 'Content-Type': 'application/json'}

    def query(payload):
    	response = requests.post(API_URL, headers=headers, json=payload)
    	return response.json()
	
    output = query({
    	"inputs": "<s>[INST] write short story about " + senario + "[/INST]",
        "parameters": {"max_new_tokens":200},
    })
    print(output[0]["generated_text"])
    return output[0]["generated_text"]

In [35]:
story = new_story(senario)

<s>[INST] write short story about there are coffee bags and a cup of coffee on a table[/INST]
There were coffee bags hanging from the ceiling, each filled with a different type of coffee. The air was thick with the aroma of freshly brewed coffee. On a table in the corner, a cup of coffee sat waiting to be enjoyed.


In [36]:
type(story)

str

In [46]:
import re

# Define a regular expression pattern to match text between <s> and [/INST]
pattern = r'<s>.*?\[/INST\]'

# Use re.sub() to remove matched text
story = re.sub(pattern, '', str(story))
story

'\nThere were coffee bags hanging from the ceiling, each filled with a different type of coffee. The air was thick with the aroma of freshly brewed coffee. On a table in the corner, a cup of coffee sat waiting to be enjoyed.'

In [47]:
# txt to speech
def text2speech(message):
    API_URL = "https://api-inference.huggingface.co/models/espnet/kan-bayashi_ljspeech_vits"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_TOKEN}"}
    payloads = {
        "inputs": message
    }

    response = requests.post(API_URL, headers=headers, json=payloads)
    with open('audio1.flac', 'wb') as file:
        file.write(response.content)

In [48]:
#senario = img2txt("maryna-vasylieva-vasilievame-2xDIrOkhEb0-unsplash.jpg")
text2speech(story)

In [ ]:
def main():
    st.set_page_config(page_title="Image to audio story", page_icon="🤖")

    st.header("turn image into audio story")
    uploaded_file = st.file_uploader("Choose an image...", type="jpg")

    if uploaded_file is not None:
        print(uploaded_file)
        bytes_data = uploaded_file.getvalue()
        with open(uploaded_file.name, "wb") as file:
            file.write(bytes_data)
        st.image(uploaded_file, caption='Uploaded Image.',
                 use_column_width=True)
        senario = img2txt(uploaded_file.name)
        story = new_story(senario)
        text2speech(story)

        with st.expander("senario"):
            st.write(senario)
        with st.expander("story"):
            st.write(story)

        st.audio("audio1.flac")

if __name__ == '__main__':
    main()